In [1]:
# Import Dependencies

import requests
import json
import pandas as pd
import numpy as np
import datetime
from config import api_key
from config import google_key
import time

from config import password
from config import username
import psycopg2
import sqlalchemy
import urllib
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import PrimaryKeyConstraint
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, Float, DateTime
from sqlalchemy.schema import Sequence

print(f'Dependencies imported...',flush=True)
print('---------------',flush=True)

Dependencies imported...
---------------


In [2]:
# Download 1000 restaurants from Yelp API with Minneapolis as the search parameter.

data = []

headers = {'Authorization': 'Bearer %s' % api_key}

url='https://api.yelp.com/v3/businesses/search'

print('Downloading Yelp Data...',flush=True)

for offset in range(0, 1000, 50):
    
    params = {
        'limit':50, 
        'location':'Minneapolis, MN',

        'categories':'restaurants',
        'offset':offset
        }  
    
    response=requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data += response.json()['businesses']
    elif response.status_code == 400:
        print('400 Bad Request')
        break
        
print(f'Yelp data downloaded...  There are {len(data)} records...',flush=True)
print('---------------',flush=True)

Yelp data downloaded...  There are 1000 records...
---------------


In [3]:
i=0
yelp_list=[]
for places in data:
    yelp_id=data[i]['id']
    name=data[i]['name']
    image=data[i]['image_url']
    categories = []
    for category in data[i]['categories']:
        cat = category['title']
        categories.append(cat)
    url=data[i]['url']
    transactions=data[i]['transactions']
    city=data[i]['location']['city']
#     price=data[i]['price']
    phone=data[i]['display_phone']
    address= data[i]['location']['display_address']
    rating=data[i]['rating']
    reviews=data[i]['review_count']
    latitude=data[i]['coordinates']['latitude']
    longitude=data[i]['coordinates']['longitude']
    if data[i]['is_closed']==False and city=="Minneapolis":
        business_dict={"yelpid":yelp_id,"name":name,"image":image,"url":url,"latitude":latitude,"longitude":longitude,"phone":phone,"categories":categories,"transactions":transactions,"address":' '.join(map(str, address)),"rating":rating,"reviews":reviews}
        yelp_list.append(business_dict)
    i+=1

print('yelp_list with needed data has been built.',flush=True)
print('---------------',flush=True)

yelp_list with needed data has been built.
---------------


In [4]:
yelp_df=pd.DataFrame(yelp_list)
yelp_df=yelp_df[['yelpid','name','image','url','latitude','longitude','address','phone','categories','transactions','rating','reviews']]
yelp_df = yelp_df.drop_duplicates(subset=['name','address'])

print('Yelp DataFrame now stored in memory as "yelp_df"',flush=True)
print(f'Removed duplicates and restaurants outside of Minneapolis. Leaving {len(yelp_df)} restaurants.',flush=True)
print('---------------',flush=True)

Yelp DataFrame now stored in memory as "yelp_df"
Removed duplicates and restaurants outside of Minneapolis. Leaving 816 restaurants.
---------------


In [ ]:
print('Matching Yelp data list to Google API...   This will take some time, as we match each record...',flush=True)

url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'
google_data=[]

for index,row in yelp_df.iterrows():
    
    params = {
        'key':google_key,
        'input':row['name'],
        'inputtype':'textquery',
        'locationbias': 'point:' + str(row['latitude']) + ", " + str(row['longitude']),
        'radius': 10,
        'fields':'place_id,name,formatted_address,geometry,rating,user_ratings_total,price_level,photos,icon'
        }
    
    response = requests.get(url, params=params)

    if len(response.json()['candidates'])>0:
        google_data.append(response.json()['candidates'][0])
    else:
        google_data.append("")
        
   print("Restaurants Remaining: {:3}".format(len(yelp_df)-index), end="\r",flush=True)

print(f'Google match has been completed...  There are {len(google_data)} records',flush=True)


In [ ]:
google_data


In [ ]:
i=0
google_list=[]

for places in google_data:
    if places != "":
        if "place_id" in places:
            google_id = places['place_id']
        if "icon"in places:
            icon=places['icon']
        photos=[]
        if "photos" in places:
            for photo in places['photos']:
                item = photo['html_attributions']
                photos.append(item)
        if "price_level" in places:
            price_level=places['price_level']
        if "name" in places:
            name = places['name']
        if "formatted_address" in places:
            address = places['formatted_address']
        if "rating" in places:
            rating  = places['rating']
        if "user_ratings_total" in places:
            reviews = places['user_ratings_total']
        if "geometry" in places:
            latitude = places['geometry']['location']['lat']
            longitude = places['geometry']['location']['lng']
        business_dict = {"googleplacesid":google_id,"icon":icon,"photos":photos,"name":name,"latitude":latitude,"longitude":longitude,"address":address,"rating":rating,"reviews":reviews,"price":price_level}
    
    else:
        business_dict = {"googleplacesid":"","icon":"","photos":"","name":"","latitude":"","longitude":"","address":"", "rating":"","reviews":"","price":""}
    
    google_list.append(business_dict)
    
    i+=1
    
print('google_list with needed data has been built.',flush=True)google_list

In [ ]:
google_df=pd.DataFrame(google_list)
google_df=google_df[google_df.name != ""]
google_df = google_df.drop_duplicates(subset=['googleplacesid'])

google_df=google_df[['googleplacesid','name','latitude','longitude','address','rating','reviews','price','icon','photos']]

google_df.to_csv('DataFiles/googledata.csv')


print('Google DataFrame now stored in memory as "google_df".',flush=True)
print(f'Removed null entries.  {len(google_df)} restaurants remain.',flush=True)
print('---------------',flush=True)

In [ ]:
i = 0
compare_list=[]
yelpgeo_list=[]

for i in range(len(google_list)):

    compare = {"Yelp":yelp_list[i]['name'],"Google":google_list[i]['name'],"GoogleAddress":google_list[i]['address'],"Yelp Address":yelp_list[i]['address']}
    compare_list.append(compare)
    i+=1

compare_df = pd.DataFrame(compare_list)
compare_df.to_csv('DataFiles/compare.csv')

print('"compare_df" has been stored in memory and csv "compare.csv" has been saved in DataFiles folder to allow easy comparison between Yelp and Google data.',flush=True)
print('---------------',flush=True)

In [5]:
print('Matching Yelp data list to Minneapolis Health Inspection API...   This will take some time, as we match each record...',flush=True)

inspection_data=[]

for index,row in yelp_df.iterrows():

    biz = row['name']

    biz_string = biz.split(' ',1)[0].upper()
    biz_string = biz_string.replace("'","")
    biz_string = biz_string.replace("&","")

    minlat=row['latitude']-.0015
    maxlat=row['latitude']+.0015
    minlon=row['longitude']-.0015
    maxlon=row['longitude']+.0015
    
    url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Food_Inspections/FeatureServer/0/query?'
    params = f"where=BusinessName%20like%20'%25{biz_string}%25'%20AND%20Latitude%20%3E%3D%20{minlat}%20AND%20Latitude%20%3C%3D%20{maxlat}%20AND%20Longitude%20%3E%3D%20{minlon}%20AND%20Longitude%20%3C%3D%20{maxlon}"
    outfields = "&outFields=BusinessName,HealthFacilityIDNumber,FullAddress,InspectionType,DateOfInspection,InspectionIDNumber,InspectionScore,Latitude,Longitude,FoodCodeText,ViolationPoints,InspectionResult,FoodCodeItem,InspectorComments,ViolationStatus,ViolationPriority&returnGeometry=false&outSR=4326"
    json = '&f=json'

    full_url = url+params+outfields+json

    response = requests.get(full_url)
    
    if response !="":
        inspection_data += response.json()['features']
        
    print("Restaurants Remaining: {:3}".format(len(yelp_df)-index), end="\r",flush=True)
    
print(f'Inspection data match has been completed...  There are {len(inspection_data)} records',flush=True)
print('---------------',flush=True)

Matching Yelp data list to Minneapolis Health Inspection API...   This will take some time, as we match each record...
Inspection data match has been completed...  There are 23998 records
---------------


In [6]:
# inspection_data = inspection_data.sort(key=operator.itemgetter(attributes[DateOfInspection]))
# inspection_data=inspection_data.attributes
inspection_data

[{'attributes': {'BusinessName': 'BUTCHER & THE BOAR',
   'HealthFacilityIDNumber': 'LIC50791',
   'FullAddress': '1121 HENNEPIN AVE',
   'InspectionType': 'Routine',
   'DateOfInspection': 1506361500000,
   'InspectionIDNumber': 59767,
   'InspectionScore': 92,
   'Latitude': 44.97466,
   'Longitude': -93.27972,
   'FoodCodeText': None,
   'ViolationPoints': 0,
   'InspectionResult': 'Complete',
   'FoodCodeItem': None,
   'InspectorComments': None,
   'ViolationStatus': None,
   'ViolationPriority': None}},
 {'attributes': {'BusinessName': 'BUTCHER & THE BOAR',
   'HealthFacilityIDNumber': 'LIC50791',
   'FullAddress': '1121 HENNEPIN AVE',
   'InspectionType': 'Routine',
   'DateOfInspection': 1506361500000,
   'InspectionIDNumber': 59767,
   'InspectionScore': 92,
   'Latitude': 44.97466,
   'Longitude': -93.27972,
   'FoodCodeText': 'Clean surfaces contacting food that is not TCS: 1. at any time when contamination may have occurred; 2. at least once every 24 hours for iced tea disp

In [7]:
inspection_data_list = []

for records in inspection_data:
    item = records['attributes']
    item['DateOfInspection']=time.strftime('%Y/%m/%d',time.gmtime(records['attributes']['DateOfInspection']/1000))
    inspection_data_list.append(item)
    
print('inspection_data_list with needed data has been built.',flush=True)
print('---------------',flush=True)

inspection_data_list with needed data has been built.
---------------


In [8]:
inspection_data_list

[{'BusinessName': 'BUTCHER & THE BOAR',
  'HealthFacilityIDNumber': 'LIC50791',
  'FullAddress': '1121 HENNEPIN AVE',
  'InspectionType': 'Routine',
  'DateOfInspection': '2017/09/25',
  'InspectionIDNumber': 59767,
  'InspectionScore': 92,
  'Latitude': 44.97466,
  'Longitude': -93.27972,
  'FoodCodeText': None,
  'ViolationPoints': 0,
  'InspectionResult': 'Complete',
  'FoodCodeItem': None,
  'InspectorComments': None,
  'ViolationStatus': None,
  'ViolationPriority': None},
 {'BusinessName': 'BUTCHER & THE BOAR',
  'HealthFacilityIDNumber': 'LIC50791',
  'FullAddress': '1121 HENNEPIN AVE',
  'InspectionType': 'Routine',
  'DateOfInspection': '2017/09/25',
  'InspectionIDNumber': 59767,
  'InspectionScore': 92,
  'Latitude': 44.97466,
  'Longitude': -93.27972,
  'FoodCodeText': 'Clean surfaces contacting food that is not TCS: 1. at any time when contamination may have occurred; 2. at least once every 24 hours for iced tea dispensers and consumer self-service utensils; 3. before rest

In [9]:
inspections_df_base = pd.DataFrame(inspection_data_list)

inspections_df_1 = inspections_df_base[['InspectionIDNumber','DateOfInspection','BusinessName','FullAddress','InspectionType','InspectionScore','Latitude','Longitude']]
inspections_df_1 = inspections_df_1.drop_duplicates(subset='InspectionIDNumber', keep='first')
inspections_df_1 = inspections_df_1.sort_values(by=['BusinessName','DateOfInspection'])
inspections_df_1 = inspections_df_1.rename(columns={'BusinessName':'businessname','FullAddress':'fulladdress','Latitude':'latitude','Longitude':'longitude','InspectionIDNumber':'inspectionidnumber','DateOfInspection':'dateofinspection','InspectionScore':'inspectionscore','InspectionType':'inspectiontype'})

inspections_df_2 = inspections_df_base[['DateOfInspection','InspectionIDNumber','BusinessName','FullAddress','InspectionType','InspectionScore','InspectionResult','FoodCodeItem','FoodCodeText','InspectorComments','ViolationPriority','ViolationStatus','ViolationPoints']]
inspections_df_2 = inspections_df_2.sort_values(by=['BusinessName','DateOfInspection'])
inspections_df_2 = inspections_df_2.rename(columns={'InspectionIDNumber':'inspectionidnumber','DateOfInspection':'dateofinspection','BusinessName':'businessname','FullAddress':'fulladdress','InspectionType':'inspectiontype','InspectionScore':'inspectionscore','InspectionResult':'inspectionresult','FoodCodeItem':'foodcodeitem','FoodCodeText':'foodcodetext','InspectorComments':'inspectorcomments','ViolationPriority':'violationpriority','ViolationStatus':'violationstatus','ViolationPoints':'violationpoints'})

inspections_df_1

,inspectionidnumber,dateofinspection,businessname,fulladdress,inspectiontype,inspectionscore,latitude,longitude
19666,44314,2017/01/27,1010 WASHINGTON WINE & SPIRITS,1010 WASHINGTON AVE S,Routine,96,44.97659,-93.25502
19669,189485,2019/02/06,1010 WASHINGTON WINE & SPIRITS,1010 WASHINGTON AVE S,Routine,96,44.97659,-93.25502
78,44334,2017/08/31,112 EATERY,112 3RD ST N,Routine,80,44.98266,-93.27156
111,224449,2017/10/06,112 EATERY,112 3RD ST N,Follow-Up,96,44.98266,-93.27156
102,224448,2018/08/31,112 EATERY,112 3RD ST N,Routine,84,44.98266,-93.27156
121,270458,2018/10/01,112 EATERY,112 3RD ST N,Follow-Up,100,44.98266,-93.27156
116,270457,2019/08/30,112 EATERY,112 3RD ST N,Routine,94,44.98266,-93.27156
21825,44398,2017/10/10,1ST WOK LAKESIDE,3236 LAKE ST W,Routine,96,44.94913,-93.32083
21858,228784,2017/12/12,1ST WOK LAKESIDE,3236 LAKE ST W,Follow-Up,100,44.94913,-93.32083
21833,228783,2018/10/18,1ST WOK LAKESIDE,3236 LAKE ST W,Routine,72,44.94913,-93.32083


In [10]:
inspections_df_2

,dateofinspection,inspectionidnumber,businessname,fulladdress,inspectiontype,inspectionscore,inspectionresult,foodcodeitem,foodcodetext,inspectorcomments,violationpriority,violationstatus,violationpoints
19666,2017/01/27,44314,1010 WASHINGTON WINE & SPIRITS,1010 WASHINGTON AVE S,Routine,96,Complete,None,None,None,None,None,0
19667,2017/01/27,44314,1010 WASHINGTON WINE & SPIRITS,1010 WASHINGTON AVE S,Routine,96,Complete,MN Rule 4626.1085A,Provide a pressure type vacuum breaker device ...,2/27/15 INSPECTION: HOSE AT MOP SINK OBSERVED ...,Priority1,Observed,4
19668,2017/01/27,44314,1010 WASHINGTON WINE & SPIRITS,1010 WASHINGTON AVE S,Routine,96,Complete,MN Rule 4626.1540,Hang mops to dry after each use and do not sto...,OBSERVED MOP STORED IN MOP BUCKET BY MOP SINK....,Priority3,Observed,0
19669,2019/02/06,189485,1010 WASHINGTON WINE & SPIRITS,1010 WASHINGTON AVE S,Routine,96,Complete,None,None,None,None,None,0
19670,2019/02/06,189485,1010 WASHINGTON WINE & SPIRITS,1010 WASHINGTON AVE S,Routine,96,Complete,MN Rule 4626.0795A,Maintain the temperature at the manifold of th...,_________________________ Observed the high-te...,Priority2,Observed,2
19671,2019/02/06,189485,1010 WASHINGTON WINE & SPIRITS,1010 WASHINGTON AVE S,Routine,96,Complete,MN Rule 4626.0710B,"Provide a readily accessible, irreversible reg...",________________________ There is a high-tempe...,Priority2,Observed,2
78,2017/08/31,44334,112 EATERY,112 3RD ST N,Routine,80,Complete,MN Rule 4626.0395A2,Maintain all cold TCS foods at 41 degrees F (5...,Many observations of potentially hazardous foo...,Priority1,Observed,4
79,2017/08/31,44334,112 EATERY,112 3RD ST N,Routine,80,Complete,MN Rule 4626.0725,EXPIRED: Secure carbon dioxide and bottle gas...,Carbon dioxide tanks in lower level observed n...,Priority3,Observed,0
80,2017/08/31,44334,112 EATERY,112 3RD ST N,Routine,80,Complete,MN Rule 4626.0735AB,All equipment and components must be in good r...,The following equipment was observed not worki...,Priority3,Observed,0
81,2017/08/31,44334,112 EATERY,112 3RD ST N,Routine,80,Complete,MN Rule 4626.1480B,Provide lockers or other suitable facilities f...,"Shoes, backpacks and other personal items obse...",Priority3,Observed,0


In [11]:
inspect_by_biz=inspections_df_1.groupby(['businessname','fulladdress','latitude','longitude'],sort=False,as_index=False).aggregate(lambda x: list(x))

print('Inspections DataFrame now stored in memory as "inspect_by_biz" and csv "InspectionsData.csv" has been saved in DataFiles folder.')
print(f'There are {len(inspections_df_1)} inspections for {len(inspect_by_biz)} facilities.')
print('---------------')

Inspections DataFrame now stored in memory as "inspect_by_biz" and csv "InspectionsData.csv" has been saved in DataFiles folder.
There are 3034 inspections for 672 facilities.
---------------


In [12]:
inspection_detail=inspections_df_2.groupby(['inspectionidnumber','dateofinspection','businessname','fulladdress','inspectiontype','inspectionscore','inspectionresult'],sort=False,as_index=False).aggregate(lambda x: list(x))

print('Inspection Detail DataFrame now stored in memory as "inspection_detail"',flush=True)

print('---------------',flush=True)

Inspection Detail DataFrame now stored in memory as "inspection_detail"
---------------


In [13]:
inspection_detail

,inspectionidnumber,dateofinspection,businessname,fulladdress,inspectiontype,inspectionscore,inspectionresult,foodcodeitem,foodcodetext,inspectorcomments,violationpriority,violationstatus,violationpoints
0,44314,2017/01/27,1010 WASHINGTON WINE & SPIRITS,1010 WASHINGTON AVE S,Routine,96,Complete,"[None, MN Rule 4626.1085A, MN Rule 4626.1540]","[None, Provide a pressure type vacuum breaker ...","[None, 2/27/15 INSPECTION: HOSE AT MOP SINK OB...","[None, Priority1, Priority3]","[None, Observed, Observed]","[0, 4, 0]"
1,189485,2019/02/06,1010 WASHINGTON WINE & SPIRITS,1010 WASHINGTON AVE S,Routine,96,Complete,"[None, MN Rule 4626.0795A, MN Rule 4626.0710B]","[None, Maintain the temperature at the manifol...","[None, _________________________ Observed the ...","[None, Priority2, Priority2]","[None, Observed, Observed]","[0, 2, 2]"
2,44334,2017/08/31,112 EATERY,112 3RD ST N,Routine,80,Complete,"[MN Rule 4626.0395A2, MN Rule 4626.0725, MN Ru...",[Maintain all cold TCS foods at 41 degrees F (...,[Many observations of potentially hazardous fo...,"[Priority1, Priority3, Priority3, Priority3, P...","[Observed, Observed, Observed, Observed, Obser...","[4, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, ..."
3,224449,2017/10/06,112 EATERY,112 3RD ST N,Follow-Up,96,Complete,"[None, MN Rule 4626.0395A2, MN Rule 4626.0390A...","[None, Maintain all cold TCS foods at 41 degre...","[None, All coolers (in kitchen, upper level ba...","[None, Priority1, Priority2, Priority3, Priori...","[None, Observed, Corrected on-site, Observed, ...","[0, 4, 2, 0, 0]"
4,224448,2018/08/31,112 EATERY,112 3RD ST N,Routine,84,Complete,"[None, MN Rule 4626.0395A2, MN Rule 4626.1515,...","[None, Maintain all cold TCS foods at 41 degre...","[None, A container of seasoned butter with roa...","[None, Priority1, Priority3, Priority3, Priori...","[None, Observed, Observed, Observed, Observed,...","[0, 4, 0, 0, 4, 4, 4, 0, 0]"
5,270458,2018/10/01,112 EATERY,112 3RD ST N,Follow-Up,100,Complete,[None],[None],[None],[None],[None],[0]
6,270457,2019/08/30,112 EATERY,112 3RD ST N,Routine,94,Complete,"[MN Rule 4626.0033A, MN Rule 4626.0710B, MN Ru...",[Employ a certified food protection manager (C...,[- Several staff have completed the certified ...,"[Priority3, Priority2, Priority2, Priority2, P...","[Observed, Observed, Observed, Observed, Obser...","[0, 2, 2, 2, 0]"
7,44398,2017/10/10,1ST WOK LAKESIDE,3236 LAKE ST W,Routine,96,Complete,"[None, MN Rule 4626.0620A, MN Rule 4626.0500B,...","[None, Provide a temperature measuring device ...","[None, Missing a thermometer in the refrigerat...","[None, Priority3, Priority1, Priority3, Priori...","[None, Corrected on-site, Observed, Observed, ...","[0, 0, 4, 0, 0, 0, 0, 0]"
8,228784,2017/12/12,1ST WOK LAKESIDE,3236 LAKE ST W,Follow-Up,100,Complete,[None],[None],[None],[None],[None],[0]
9,228783,2018/10/18,1ST WOK LAKESIDE,3236 LAKE ST W,Routine,72,Complete,"[MN Rule 4626.1070A, MN Rule 4626.1040B, MN Ru...","[Provide at least 1 handwashing sink, or the n...",[\r\r OBSERVED ONE HAND WASHING SINK FOR THE ...,"[Priority2, Priority3, Priority1, Priority2, P...","[Observed, Observed, Observed, Observed, Obser...","[2, 0, 4, 2, 4, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, ..."


In [14]:
#Postgres username, password, and database name
ipaddress = 'localhost'
port = '5432'
username = username
password = password 
dbname = 'Minneapolis_Restaurants'
# A long string that contains the necessary Postgres login information
postgres_str = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'

In [ ]:
# Creates Classes which will serve as the anchor points for our Table, loads table to Postgres and uplads the data

Base = declarative_base()
engine = create_engine(postgres_str)

class YelpData(Base):
    __tablename__ = 'yelpdata'
    index=Column(Integer,primary_key=True,autoincrement=True)
    yelpid=Column(String,nullable=False)
    name=Column(String)
    image=Column(String)
    url=Column(String)
    latitude=Column(Float(20))
    longitude=Column(Float(20))
    address=Column(String)
    phone=Column(String)
    categories=Column(String)
    transactions=Column(String)
    rating=Column(Float(10))
    reviews=Column(Integer)
                   
Base.metadata.create_all(engine)

yelp_df.to_sql('yelpdata', engine, if_exists='replace', index=True)

print(f'Table "yelpdata" uploaded to postgreSQL database "Minneapolis_Restaurants".',flush=True)
print('---------------')

In [ ]:
# Creates Classes which will serve as the anchor points for our Table, loads table to Postgres and uplads the data

Base = declarative_base()
engine = create_engine(postgres_str)

class GoogleData(Base):
    __tablename__ = 'googledata'
    googleplacesid=Column(String,primary_key=True, nullable=False)
    name=Column(String)
    latitude=Column(Float(20))
    longitude=Column(Float(20))
    address=Column(String)
    rating=Column(Float(10))
    reviews=Column(Integer) 
    price=Column(Integer)
    icon=Column(String)
    photos=Column(String)
                   
Base.metadata.create_all(engine)

google_df.to_sql('googledata', engine, if_exists='replace', index=True)

print(f'Table "googledata" uploaded to postgreSQL database "Minneapolis_Restaurants".',flush=True)
print('---------------',flush=True)

In [15]:
# Creates Classes which will serve as the anchor points for our Table, loads table to Postgres and uplads the data

Base = declarative_base()
engine = create_engine(postgres_str)

class InspectionsData(Base):
    __tablename__ = 'inspectionsdata'
    index=Column(Integer,primary_key=True,autoincrement=True)
    businessname=Column(String,nullable=False)
    fulladdress=Column(String)
    healthfacilityidnumber=Column(String)
    latitude=Column(Float(20))
    longitude=Column(Float(20))
    inspectionidnumber=Column(String)
    dateofinspection=Column(String)
    inspectionscore=Column(String)
    inspectiontype=Column(String)
                   
Base.metadata.create_all(engine)

inspect_by_biz.to_sql('inspectionsdata', engine, if_exists='replace', index=True)

print(f'Table "inspectionsdata" uploaded to postgreSQL database "Minneapolis_Restaurants".',flush=True)
print('---------------',flush=True)

Table "inspectionsdata" uploaded to postgreSQL database "Minneapolis_Restaurants".
---------------


In [16]:
# Creates Classes which will serve as the anchor points for our Table, loads table to Postgres and uplads the data

Base = declarative_base()
engine = create_engine(postgres_str)

class InspectionsData(Base):
    __tablename__ = 'inspectionsdata'
    inspectionidnumber=Column(String,primary_key=True)
    dateofinspection=Column(String)
    businessname=Column(String)
    fulladdress=Column(String)
    inspectiontype=Column(String)
    inspectionscore=Column(String)
    inspectionresult=Column(String)
    foodcodeitem=Column(String)
    foodcodetext=Column(String)
    inspectorcomments=Column(String)
    violationpriority=Column(String)
    violationstatus=Column(String)
    violationpoints=Column(String)
                   
Base.metadata.create_all(engine)

inspection_detail.to_sql('inspectionsdetail', engine, if_exists='replace', index=True)



print(f'Table "inspectionsdetail" uploaded to postgreSQL database "Minneapolis_Restaurants".',flush=True)
print('---------------',flush=True)
print("DONE.  Don't forget to fix the SQL data types! Use the DataTypeChange script to fix your Minneapolis_Restaurants DB",flush=True)

Table "inspectionsdetail" uploaded to postgreSQL database "Minneapolis_Restaurants".
---------------
DONE.  Don't forget to fix the SQL data types! Use the DataTypeChange script to fix your Minneapolis_Restaurants DB
